# 100 Prepare input scenario database
In this notebook, we will prepare the input scenario database.

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
AR6_FILTERS = {
    'IMP_marker':[
        'SP', 'LD', 'Ren'
    ],
    #'Category':['C1', 'C2', 'C3']
}

<IPython.core.display.Javascript object>

In [3]:
import pyam
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import dotenv
import os

from utils import calculate_netzero

import sys
sys.path.append('../scripts')
from cdr import compile_novel_cdr_estimates

<IPython.core.display.Javascript object>

/Users/gauravganti/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


<IPython.core.display.Javascript object>

In [4]:
dotenv.load_dotenv()

True

<IPython.core.display.Javascript object>

## Read in the input dataframe
The file should be in the `data` folder and is called: `20220314_ar6emissions_harmonized_infilled.csv`

In [5]:
df_input = pyam.IamDataFrame(
    Path(
        '../data/20220314_ar6emissions_harmonized_infilled.csv'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/20220314_ar6emissions_harmonized_infilled.csv


<IPython.core.display.Javascript object>

For now, we will also add in the metadata that comes along with the AR6 scenario database.

In [6]:
df_input.load_meta(
    Path(
        os.environ['AR6_META']
    )
)

pyam.core - WARNING: Reading meta indicators for 1703 out of 1836 scenarios, ignoring 601 scenarios from file


<IPython.core.display.Javascript object>

We will also need the original scenario database for the CDR variables.

In [7]:
df_ar6 = pyam.IamDataFrame(
    Path(
        os.environ['AR6_GLOBAL']
    )
)

pyam.core - INFO: Reading file /Users/gauravganti/Documents/datasets/AR6_Scenarios_Database_World_v1.1.csv


<IPython.core.display.Javascript object>

In [8]:
df_ar6.load_meta(
    Path(
        os.environ['AR6_META']
    )
)

pyam.core - WARNING: Reading meta indicators, ignoring 6 scenarios from file


<IPython.core.display.Javascript object>

## Data processing

Step 1: Filter out the necessary scenarios

In [9]:
df_proc = df_input.filter(
    **AR6_FILTERS
)

<IPython.core.display.Javascript object>

Step 2: Check if total CO2 emissions are reported.

In [10]:
df_proc.require_variable(
    'AR6 climate diagnostics|Infilled|Emissions|CO2'
)

/var/folders/nz/kr7bwztj0_d9xrfz1jd4zcdh0000gp/T/ipykernel_3937/255023924.py:1: DeprecationWarning: This method is deprecated and will be removed in future versions. Use `df.require_data()` instead.
  df_proc.require_variable(
pyam.core - INFO: 3 scenarios do not include required variable `AR6 climate diagnostics|Infilled|Emissions|CO2`


,model,scenario
0,MESSAGEix-GLOBIOM 1.0,LowEnergyDemand_1.3_IPCC
1,REMIND-MAgPIE 2.1-4.2,SusDev_SDP-PkBudg1000
2,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_ HighRE_Budg900


<IPython.core.display.Javascript object>

Step 3: It appears not. So let us construct this variable.

In [11]:
df_proc.aggregate(
    variable='AR6 climate diagnostics|Infilled|Emissions|CO2',
    components=[
        'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
        'AR6 climate diagnostics|Infilled|Emissions|CO2|AFOLU'
    ],
    append=True
)

<IPython.core.display.Javascript object>

Step 4: Assign the year of net zero CO2. It should already be present in the metadata, but it won't hurt to recompute it.

In [12]:
co2 = (
    df_proc
    .filter(
        variable='AR6 climate diagnostics|Infilled|Emissions|CO2'
    )
    .swap_time_for_year()
    .timeseries()
)

<IPython.core.display.Javascript object>

In [13]:
df_proc.set_meta(
    calculate_netzero(co2),
    'netzero|CO2'
)

<IPython.core.display.Javascript object>

Step 5: Filter out for the novel CDR variables and the necessary scenarios in the AR6 scenario database.

In [14]:
novel_cdr = compile_novel_cdr_estimates(
    df_ar6,
    AR6_FILTERS
)

{'novel_cdr_vars': ['Carbon Sequestration|Direct Air Capture', 'Carbon Sequestration|Enhanced Weathering', 'Carbon Sequestration|CCS|Biomass'], 'netzero_column': 'netzero|CO2'}
['Carbon Sequestration|CCS|Biomass', 'Carbon Sequestration|Direct Air Capture', 'Carbon Sequestration|Enhanced Weathering']


<IPython.core.display.Javascript object>

## Save out the data

In [15]:
df_proc.to_csv(
    Path(
        '../data/100_scenarios.csv'
    )
)

<IPython.core.display.Javascript object>

In [16]:
df_proc.export_meta(
    Path(
        '../data/100_meta.xlsx'
    )
)

<IPython.core.display.Javascript object>

In [17]:
novel_cdr.to_csv(
    Path(
        '../data/100_novel_cdr.csv'
    )
)

<IPython.core.display.Javascript object>

Lastly, we also want to read out the scenarios list.

In [18]:
(
    df_proc
    .meta
    .index
    .to_frame()
    .reset_index(drop=True)
    .to_csv(
        Path(
            '../data/100_scenario_names.csv'
        )
    )
)

<IPython.core.display.Javascript object>